In [23]:
# Dependencies
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import folium

# API
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         faya  18.3851   42.4509     20.29        68          17   
1        1      farsund  58.0948    6.8047     10.53        90         100   
2        2  new norfolk -42.7826  147.0587     12.18        96          99   
3        3    jamestown  42.0970  -79.2353     16.12        52          75   
4        4      lanzhou  36.0564  103.7922     20.59        31          58   

   Wind Speed Country        Date  
0        1.72      SA  1715278691  
1        4.88      NO  1715278692  
2        0.75      AU  1715278692  
3        4.63      US  1715278682  
4        8.25      CN  1715278694

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [25]:
# Map centered on 0,0
m = folium.Map(location=[0, 0], zoom_start=2, scrollWheelZoom=False) # <--- stops zooming (stops panning and zooming with mouse)

# Markers
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 18,  # This divides the humidity by a fixed number to determine the size of the dots.
        popup=row['City'],
        fill=True,
        color='blue',
        fill_opacity=0.7
    ).add_to(m)

# Remove zoom controls
#m.options['zoomControl'] = False # <--- stops zooming (removes zoom control; cool to have so I disabled this line)

# Display the map
m


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Drop nulls
city_data_df_clean = city_data_df.dropna().reset_index(drop=True)


# Clear humidity > 45, Max Temp > 25, Wind Speed > 6, Cloudiness > 50
city_data_df_clean = city_data_df_clean[(city_data_df_clean['Humidity'] <= 45) & 
                                        (city_data_df_clean['Max Temp'] <= 25) & 
                                        (city_data_df_clean['Wind Speed'] <= 6) & 
                                        (city_data_df_clean['Cloudiness'] <= 50)]


# Display sample data
city_data_df_clean

City_ID                 City      Lat       Lng  Max Temp  Humidity  \
66        66        mar del plata -38.0023  -57.5575     17.19        35   
90        90         saint george  37.1041 -113.5841     21.90        18   
115      115           cedar city  37.6775 -113.0619     12.44        38   
161      161          yellowknife  62.4560 -114.3525     16.29        29   
172      172          danielskuil -28.1887   23.5395     20.02        30   
176      176           challapata -18.9000  -66.7667     16.31        32   
178      178              neepawa  50.2289  -99.4664     20.22        38   
181      181                 hovd  48.0056   91.6419     13.00        35   
185      185              shakawe -18.3667   21.8500     21.46        24   
221      222            hay river  60.8156 -115.7999     15.99        36   
299      301             ulaangom  49.9811   92.0667     12.51        41   
303      305              kingman  35.1894 -114.0530     21.18        11   
311      313               tandil -37.3217  -59.1332     15.94        36   
337      339             kamloops  50.6665 -120.3192     19.76        32   
343      345               zaysan  47.4667   84.8667     17.57        41   
357      359                korla  41.7597   86.1469     20.89        18   
373      375         port hedland -20.3167  118.5667     20.03        40   
434      437  veinticinco de mayo -35.4323  -60.1727     15.52        37   
461      464              vryburg -26.9566   24.7284     20.30        29   
499      502                altay  47.8667   88.1167     14.33        17   
528      531            chivilcoy -34.8957  -60.0167     15.18        42   
536      539         khudumelapye -23.8833   24.7500     18.60        23   
542      545               kalabo -14.9700   22.6814     19.33        32   

     Cloudiness  Wind Speed Country        Date  
66            0        4.47      AR  1715278820  
90            0        2.06      US  1715279048  
115           0        4.63      US  1715279108  
161          20        5.36      CA  1715279059  
172           5        4.21      ZA  1715279182  
176           7        4.27      BO  1715279187  
178          37        0.43      CA  1715279189  
181           0        2.58      MN  1715279193  
185           1        2.06      BW  1715279198  
221          20        5.66      CA  1715279246  
299           5        1.78      MN  1715279347  
303           0        4.63      US  1715279352  
311           7        2.87      AR  1715279063  
337          20        4.12      CA  1715279395  
343          18        2.31      KZ  1715279175  
357           0        1.53      CN  1715279422  
373           0        2.57      AU  1715279443  
434           0        3.59      AR  1715279524  
461           0        3.92      ZA  1715279559  
499           0        3.58      CN  1715279608  
528           0        3.39      AR  1715279645  
536           0        3.77      BW  1715279656  
542          26        2.09      ZM  1715279663

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
import requests
from requests.structures import CaseInsensitiveDict

print("Starting Hotel Search")

# Iterate through the clean DataFrame
for index, row in city_data_df_clean.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Calculate the radius of the circle filter
    radius = 10000  # Assuming a radius of 5000 meters

    # Construct the circle filter
    circle_filter = f"circle:{lng},{lat},{radius}"

    # Construct the URL for hotel search
    url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter={circle_filter}&limit=1&apiKey={geoapify_key}"

    # Set headers
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    # Send GET request
    resp = requests.get(url, headers=headers)

    # Extract hotel name from response if available
    data = resp.json()
    if "features" in data and data["features"]:
        try:
            hotel_name = data["features"][0]["properties"]["name"]
            print(f"{city_data_df_clean.loc[index, 'City']} - Nearest Hotel: {hotel_name}")
            # Add hotel name to the DataFrame
            city_data_df_clean.at[index, "Hotel Name"] = hotel_name
        except KeyError:
            # If there is an issue extracting the hotel name, set it as "Error/Unknown"
            print(f"{city_data_df_clean.loc[index, 'City']} - Error Extracting Hotel Name")
            city_data_df_clean.at[index, "Hotel Name"] = "Error/Unknown"
    else:
        print(f"{city_data_df_clean.loc[index, 'City']} - No hotel found.")
        city_data_df_clean.at[index, "Hotel Name"] = "No hotel found."

Starting Hotel Search
mar del plata - Nearest Hotel: Hotel Valles. Mar del Plata
saint george - Nearest Hotel: Quality Inn
cedar city - Nearest Hotel: Valu Inn Cedar City
yellowknife - Nearest Hotel: The Explorer Hotel
danielskuil - No hotel found.
challapata - Nearest Hotel: Residencial Marian
neepawa - Nearest Hotel: Vivian Motor Inn
hovd - Nearest Hotel: Цамбагарав зочид буудал
shakawe - Nearest Hotel: Drotsky's Cabins
hay river - Error Extracting Hotel Name
ulaangom - Nearest Hotel: Их наяд буудал
kingman - Nearest Hotel: Red Roof Inn
tandil - Nearest Hotel: Hotel "El Paraiso"
kamloops - Nearest Hotel: Scott's Inn
zaysan - Nearest Hotel: Рахат қонақүй
korla - Nearest Hotel: Silver Star Hotel
port hedland - Nearest Hotel: Hospitality Port Hedland
veinticinco de mayo - Nearest Hotel: Gran Hotel 25 de Mayo
vryburg - Nearest Hotel: Kameelboom Lodge
altay - Nearest Hotel: 金都酒店
chivilcoy - Nearest Hotel: Falcone
khudumelapye - No hotel found.
kalabo - Nearest Hotel: Golden Lodge Luxury A

In [38]:
# Display the updated DataFrame
# Drop the "Humidity", "Cloudiness", "Wind Speed", and "Date" columns from the DataFrame
#city_data_df_clean.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis=1, inplace=True)
city_data_df_clean = city_data_df_clean[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]]
city_data_df_clean

City Country      Lat       Lng  Humidity  \
66         mar del plata      AR -38.0023  -57.5575        35   
90          saint george      US  37.1041 -113.5841        18   
115           cedar city      US  37.6775 -113.0619        38   
161          yellowknife      CA  62.4560 -114.3525        29   
172          danielskuil      ZA -28.1887   23.5395        30   
176           challapata      BO -18.9000  -66.7667        32   
178              neepawa      CA  50.2289  -99.4664        38   
181                 hovd      MN  48.0056   91.6419        35   
185              shakawe      BW -18.3667   21.8500        24   
221            hay river      CA  60.8156 -115.7999        36   
299             ulaangom      MN  49.9811   92.0667        41   
303              kingman      US  35.1894 -114.0530        11   
311               tandil      AR -37.3217  -59.1332        36   
337             kamloops      CA  50.6665 -120.3192        32   
343               zaysan      KZ  47.4667   84.8667        41   
357                korla      CN  41.7597   86.1469        18   
373         port hedland      AU -20.3167  118.5667        40   
434  veinticinco de mayo      AR -35.4323  -60.1727        37   
461              vryburg      ZA -26.9566   24.7284        29   
499                altay      CN  47.8667   88.1167        17   
528            chivilcoy      AR -34.8957  -60.0167        42   
536         khudumelapye      BW -23.8833   24.7500        23   
542               kalabo      ZM -14.9700   22.6814        32   

                          Hotel Name  
66       Hotel Valles. Mar del Plata  
90                       Quality Inn  
115              Valu Inn Cedar City  
161               The Explorer Hotel  
172                  No hotel found.  
176               Residencial Marian  
178                 Vivian Motor Inn  
181          Цамбагарав зочид буудал  
185                 Drotsky's Cabins  
221                    Error/Unknown  
299                   Их наяд буудал  
303                     Red Roof Inn  
311               Hotel "El Paraiso"  
337                      Scott's Inn  
343                    Рахат қонақүй  
357                Silver Star Hotel  
373         Hospitality Port Hedland  
434            Gran Hotel 25 de Mayo  
461                 Kameelboom Lodge  
499                             金都酒店  
528                          Falcone  
536                  No hotel found.  
542  Golden Lodge Luxury Acomodation

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
# Map centered on 0,0
m = folium.Map(location=[0, 0], zoom_start=2, scrollWheelZoom=False) # <--- stops zooming (stops panning and zooming with mouse)

# Markers
for index, row in city_data_df_clean.iterrows():
    popup_text = f"City: {row['City']}, Country: {row['Country']}<br>Hotel: {row['Hotel Name']}"
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 10,  # This divides the humidity by a fixed number to determine the size of the dots.
        popup=popup_text,
        fill=True,
        color='blue',
        fill_opacity=0.7
    ).add_to(m)

# Display the map
m

